In [8]:
from PyPDF2 import PdfFileReader
import os
import json
import pandas as pd

Extract information from pdf file

In [9]:
def extract_pdf(path):
    pdf=PdfFileReader(path)
    number_pages=pdf.getNumPages()
    content=''
    for i in range(number_pages):
        content+=pdf.getPage(i).extractText()
    
    content=content.replace('\\n','. ').replace('\n\n','. ')
    content=content.replace('\n','. ')
    content=content.replace('\t',' ')
    
    return content

Get path to each pdf file stored in folder  

In [10]:

def get_path_pdf(path_folder):
    path_pdf=[]
    for dirname,_,filenames in os.walk(path_folder):
        for filename in filenames:
            path_pdf.append(os.path.join(dirname,filename))
    return path_pdf


Read JD

In [35]:
path_JD="./static/files_JD/"
path_2_pdfJD=get_path_pdf(path_JD)
content_JD=extract_pdf(path_2_pdfJD[0])


Read CV

In [36]:
path_CV="./static/files_CV/"
path_2_pdfCV=get_path_pdf(path_CV)
content_CV=[]
for p in path_2_pdfCV:
    content_CV.append(extract_pdf(p))


Using Spacy to extract skill

In [13]:
from spacy.pipeline import EntityRuler
from spacy import displacy
import jsonlines
import spacy
import re 
import pyperclip
from sympy import true

In [14]:
json_path="skill_paterns.jsonl"

with jsonlines.open(json_path) as f:
    entities=[line["label"].upper() for line in f.iter()]

# spacy model
nlp=spacy.load("en_core_web_lg")

In [15]:
ruler = nlp.add_pipe("entity_ruler")
ruler.from_disk(json_path)

PERSONAL INFORMATION

In [16]:

def get_skill(content_CV):
    doc=nlp(content_CV)
    skill_list=[]
    for ent in doc.ents:
        if ent.label_=='SKILL':
            skill_list.append(ent.text)
    skill_list=list(set(skill_list))
    return skill_list

def get_GPA(content_CV):
    doc=nlp(content_CV)
    sent=list(doc.sents)
    gpa=''
    for s in sent:
        s=s.text
        if s.find('GPA') !=-1 :
            gba_pos= s.find('GPA') 
            for i in range (gba_pos+3,gba_pos+8):
                if (s[i] >'0' and s[i]<'9' or s[i]=='.'):
                    gpa+=s[i]
    if gpa=='':
        return None
    else:
        return gpa

def get_experience(content_CV):
    '''
    Return\n
        None if no experience\n
        List of experience if exist
    '''
    doc=nlp(content_CV)
    sent=list(doc.sents)
    final_res=''
    ex=[]
    for s in sent:
        s_text=s.text.split(' ')
        flag=False
        for i in s_text:
            i=i.lower()
            if i=='experience':
                flag=True
        if flag==True:
            ex.append([s.text])
    if (len(ex)==0):
        final_res=None
    else:
        final_res=ex
    return final_res

def get_education(content_CV):
    doc=nlp(content_CV)
    sent=list(doc.sents)
    edu=''
    pos_edu=-1
    pos_edu_end=-1
    for s in sent:
        s_text=s.text.lower()
        flag=False
        if s_text.find('education')!=-1 or s_text.find('university')!=-1 or s_text.find('academic')!=-1:
            edu =s.text
            if s_text.find('education')!=-1:
                pos_edu=s_text.find('education')+9
            if s_text.find('academic') !=-1 :
                pos_edu=s_text.find('academic')
            else:
                pos_edu=s_text.find('university')
            
            pos_edu_end=s_text.find('. ',pos_edu)
            edu=edu[pos_edu:pos_edu_end]
    if edu=='':
        return None
    else:
        return edu

def get_phone(content_CV):
    '''
    Format for Phone-Numbers:
    134567890
    123-456-7890
    (123)(456)(7890)
    123.456.7890
    (123)(456-7890)
    '''
    phone= re.compile(r'''
        (\d{3}|\(\d{3}\))    
        (\s|-|\.)?           
        (\d{3}|\(\d{3}\))    
        (\s|-|\.)?           
        (\d{4}|\(\d{4}\))    
    ''', re.VERBOSE) 
    matchs=phone.finditer(content_CV)
    res = ''
    for match in matchs:
        res+=match.group(0)
    if res=='':
        return None
    else:
        return res

def get_link(link,content_CV):
    matchs=link.finditer(content_CV)
    res=''
    for match in matchs:
        res+=match.group(0)
    if res=='':
        return None
    else:
        return res

# test=content_CV[0]
# print('SKILL    :',get_skill(test))
# print('GPA      :',get_GPA(test))       
# print('Ex       :',get_experience(test))
# print()
# print('EDU      :',get_education(test))
# print('Personal Website :',get_link(web,test))
# print('GIT      :',get_link(git,test))
# print('Linkedin :',get_link(linkedin,test))
# print('Mail     :',get_link(mail,test))

# print('Phone    :',get_phone(test))


NAME

In [17]:
from spacy.tokens import Span
import dateparser
def expand_person_entities(doc):
    new_ents = []
    for ent in doc.ents:
        # Only check for title if it's a person and not the first token
        if ent.label_ == "PERSON":
            if ent.start != 0:
                # if person preceded by title, include title in entity
                prev_token = doc[ent.start - 1]
                if prev_token.text in ("Dr", "Dr.", "Mr", "Mr.", "Ms", "Ms."):
                    new_ent = Span(doc, ent.start - 1, ent.end, label=ent.label)
                    new_ents.append(new_ent)
                else:
                    # if entity can be parsed as a date, it's not a person
                    if dateparser.parse(ent.text) is None:
                        new_ents.append(ent) 
        else:
            new_ents.append(ent)
    doc.ents = new_ents
    return doc

# Add the component after the named entity recognizer
# # nlp.remove_pipe('expand_person_entities')
# @Language.component("expand_person_entities")
# nlp.add_pipe("expand_person_entities", after='ner')

# doc = nlp(content_CV[0])
# [(ent.text, ent.label_) for ent in doc.ents if ent.label_=='PERSON']

In [18]:
from spacy.matcher import Matcher
# initialize matcher with a vocab

def extract_name(content_CV):
    matcher = Matcher(nlp.vocab)
    doc=nlp(content_CV)
    for ent in doc.ents:
        if ent.label_=='PERSON':
            return(ent.text)
    
    # First name and Last name are always Proper Nouns
    pattern = [
        {'POS': 'PROPN'},{'POS': 'PROPN'},
        ]
    matcher.add('NAME',[pattern])
    matches = matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        return span.text
        
    return None

extract_name(content_CV[0])

'Alice Clark'

Check Accuracy of name

In [20]:
# train_data = json.load(open('./data/training/train_data.json','rb')) 

def name_acc(train_data):
    list_an=[]
    list_txt=[]
    name=[]

    for text,annotation in train_data:
        list_an.append(list(annotation.values()))
        list_txt.append(text)

    target_name=[]
    predic_name=[]

    similar=0
    for i in range(len(list_an)):
        t=''
        p=''
        p=extract_name(list_txt[i])
        predic_name.append(p)
        for j in range(len(list_an[i][0])):
            if list_an[i][0][j][2]=='Name':
                t=list_txt[i][list_an[i][0][j][0]:list_an[i][0][j][1]]
                target_name.append(t)
        if t==p:
            similar+=1
    return (similar/len(list_an)*100)    
# name_acc(train_data)

Load accuracy name

In [21]:
nlp_model=spacy.load('nlp_model')

Using sklearn to calculate the cosine similarity score of CV and JD

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [53]:
def cosine_similar(list_CV,JD):
    df_rank_match=pd.DataFrame(columns=['ID','Percent_Match_JD'])
    for i in range(len(list_CV)):
        test=[list_CV[i],JD]
        cv = CountVectorizer()
        count_matrix= cv.fit_transform(test)
        match=cosine_similarity(count_matrix)        
        row={
            'ID':i,
            'Percent_Match_JD': round(match[0][1]*100,2)
        }
        new_df=pd.DataFrame([row])
        df_rank_match=pd.concat([df_rank_match,new_df],axis=0,ignore_index=True)
    return df_rank_match 

Final extract and save to dataframe

In [55]:
def extract_all_data(list_content_CV,content_JD):
    df_match=cosine_similar(list_content_CV,content_JD)
    extract_df_match=df_match["Percent_Match_JD"]

    label=['ID','Name','Email','Phone_Number','Related_Link','Skill','Experience','Education','GPA']

    df_information = pd.DataFrame(columns=label) 
    # bitbucket
    mail = re.compile(r'[a-zA-Z0-9-\.]+@[a-zA-Z-\.]*\.(com|edu|net)')
    git = re.compile(r'(gitlab.com/|github.com/)+[a-zA-Z0-9-\.]*')
    web = re.compile(r'(http://|https:// )+[a-zA-Z0-9-"/"-\.]*')
    linkedin = re.compile(r'linkedin.com/+[a-zA-Z0-9-"/"-\.]*')

    for i in range(len(list_content_CV)):
        content=list_content_CV[i]
        link=[]
        link.append(get_link(git,content))
        link.append(get_link(web,content))
        link.append(get_link(linkedin,content))
        row={
                'ID':i,
                'Name':extract_name(content),
                'Email':get_link(mail,content),
                'Phone_Number':get_phone(content),
                'Related_Link':link,
                'Skill':get_skill(content),
                'Experience':get_experience(content),
                'Education':get_education(content),
                'GPA':get_GPA(content)
            }
        new_df=pd.DataFrame([row])
        df_information=pd.concat([df_information,new_df],axis=0,ignore_index=True)
    df_information=df_information.join(extract_df_match)
    df_information=df_information.sort_values(by='Percent_Match_JD', ascending=False)
    return df_information


# list_CV=[]
# count=0
# train_data = json.load(open('./data/training/train_data.json','rb')) 

# for text,annotation in train_data:
#     count+=1
#     list_CV.append(text)
#     if count==50:
#         break

df_detail=extract_all_data(content_CV,content_JD)
df_detail

,ID,Name,Email,Phone_Number,Related_Link,Skill,Experience,Education,GPA,Percent_Match_JD
0,0,Alice Clark,None,None,"[None, None, None]","[Database, database, business intelligence, Wi...",[[• 20+ y ears of experience in data han dling...,None,None,60.24
2,2,Michael Smith,None,None,"[None, None, None]","[Database, Databases, business intelligence, S...","[[10+ years of Experience in Designing, Develo...",University of Manchester - UK,None,58.20
1,1,Phan Thi Bich Ngoc Curriculum Vitae,Ptbichngoc2502@gmail.com,0347297090,"[github.com/ngoc2502., None, None]","[Computer Science, database, OpenCV, algorithm...",None,University of Science,None,40.12
